## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Mobile_Report

### Links:  

[[rep_mobile_citizens_id_city_partner]]

[[sessions_st_mobile]]

[[subscriptions_st_mobile]]


### Table

In [3]:
query_text = """--sql
CREATE TABLE db1.total_active_users_per_day_full_table
(
    `report_date` Date,
    `partner_uuid` String,
    `city` String,
    `citizen_id` Int32,
    `monetization` UInt8,
    `subscriptions_state` String,
    `address_uuid` String,
    `flat_uuid` String,
    `created_at` String,
    `activated_at` DateTime
)
ENGINE = MergeTree()
ORDER BY partner_uuid
"""

ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 57
 Code: 57. DB::Exception: Table db1.total_active_users_per_day_full_table already exists. (TABLE_ALREADY_EXISTS) (version 25.3.6.56 (official build))


In [4]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.total_active_users_per_day_full_table_mv
    REFRESH EVERY 1 DAY OFFSET 4 HOUR 10 MINUTE TO db1.total_active_users_per_day_full_table AS
       SELECT
        r_m_c.partner_uuid AS partner_uuid,
        ses_st_m.citizen_id AS citizen_id,
        r_m_c.report_date AS report_date,
        r_m_c.monetization AS monetization,
        sub_st_m.state AS subscriptions_state,
        r_m_c.city AS city,
        r_m_c.address_uuid AS address_uuid,
        r_m_c.flat_uuid AS flat_uuid,
        c_d_m.created_at AS created_at,
        r_m_c.activated_at AS activated_at
    FROM db1.rep_mobile_citizens_id_city_partner as r_m_c
    JOIN db1.`sessions_st_mobile_ch` AS ses_st_m
        ON r_m_c.citizen_id = ses_st_m.citizen_id
        AND r_m_c.report_date = ses_st_m.report_date 
    LEFT JOIN db1.`subscriptions_st_mobile_ch` AS sub_st_m
        ON ses_st_m.citizen_id = sub_st_m.citizen_id
        AND ses_st_m.report_date = sub_st_m.report_date
    LEFT JOIN db1.`citizens_dir_mobile_ch` AS c_d_m ON c_d_m.`citizen_id`  = r_m_c.`citizen_id`
    WHERE 
        toDate(`last_use`) 
        BETWEEN toStartOfMonth(r_m_c.report_date) and r_m_c.report_date 
        AND  r_m_c.state = 'activated'
    --
     SETTINGS join_algorithm = 'partial_merge'
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [5]:
query_text = """--sql
    SELECT
        *
    FROM db1.total_active_users_per_day_full_table
    ORDER BY report_date DESC
    limit 100
    """

ch.query_run(query_text)

report_date,partner_uuid,city,citizen_id,monetization,subscriptions_state,address_uuid,flat_uuid,created_at,activated_at
date,str,str,i64,i64,str,str,str,str,datetime[ns]
2025-09-02,"""""","""""",1303931,0,"""""","""""","""""","""2023-10-03 17:37:56""",1970-01-01 00:00:00
2025-09-02,"""""","""Миасс""",1303941,1,"""""","""e42cc4ed-b469-48e4-921a-94b56e…","""5a97bd8c-bd09-49f3-b984-7b57d1…","""2023-10-03 17:41:27""",2023-10-03 17:41:56
2025-09-02,"""""","""Казань""",1312145,1,"""""","""0a8c8816-af31-45f9-a3a4-090370…","""dad96e61-1054-4eea-8b52-ede76f…","""2023-10-11 20:08:10""",2023-11-03 15:50:13
2025-09-02,"""""","""""",1331970,0,"""""","""""","""""","""2023-10-31 17:43:44""",1970-01-01 00:00:00
2025-09-02,"""""","""Кохма""",1344895,1,"""""","""4008d4ea-d4cf-447d-997d-1134c8…","""c5b4e33e-5a32-4638-931b-6a96f8…","""2023-11-12 16:13:32""",2023-11-12 16:16:43
…,…,…,…,…,…,…,…,…,…
2025-09-02,"""010cd1e6-7f7b-4613-9209-96b28d…","""Коряжма""",1367125,1,"""""","""cd209cca-837b-4dc1-9786-d6b3d4…","""793a9e23-b8ce-4e20-a475-e8819c…","""2023-12-03 09:40:36""",2023-12-03 09:42:28
2025-09-02,"""010cd1e6-7f7b-4613-9209-96b28d…","""Котлас""",1381785,1,"""""","""252a6a23-3393-4704-8d07-4187f2…","""1d3b939b-622d-4d2c-a65d-5c8a36…","""2023-12-18 17:17:17""",2023-12-18 17:20:14
2025-09-02,"""010cd1e6-7f7b-4613-9209-96b28d…","""Котлас""",1381808,1,"""""","""252a6a23-3393-4704-8d07-4187f2…","""1d3b939b-622d-4d2c-a65d-5c8a36…","""2023-12-18 17:38:11""",2023-12-18 17:41:27


### delete a part


In [22]:
query_text = """--sql
    ALTER TABLE db1.total_active_users_per_day_full_table DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop mv

In [12]:
query_text = """--sql
    DROP TABLE db1.total_active_users_per_day_full_table_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [4]:
query_text = """--sql
    DROP TABLE db1.total_active_users_per_day_full_table
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.total_active_users_per_day_full_table_mv
"""

ch.query_run(query_text)